# 1. IMPORT AND SETUP TENSORFLOW

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
!pip3 install tensorflow==2.4.1
!pip3 install cython
!pip3 install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

# 2. MOUNT DRIVER

In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/artificial-intelligence')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. CREATE CSV FROM XML (LABLELING)

In [11]:
# Cover xml train to csv
!python3 workspace/discharge_record/scripts/xml_to_csv.py \
    --xml_folder /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/images/train_07_04_2021 \
    --type_data train \
    --csv_out /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations

# Cover xml test to csv
!python3 workspace/discharge_record/scripts/xml_to_csv.py \
    --xml_folder /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/images/test_07_04_2021 \
    --type_data test \
    --csv_out /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations

2021-04-07 17:43:48.844456: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Successfully converted xml to csv.
2021-04-07 17:43:51.740296: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Successfully converted xml to csv.


# 4. CREATE TF RECORD

In [12]:
# Train tf record
!python3 workspace/discharge_record/scripts/create_tf_record.py \
    --csv_input /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations/train_labels.csv \
    --image_dir /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/images/train_07_04_2021/ \
    --output_path /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations/train.record

# test tf record
!python3 workspace/discharge_record/scripts/create_tf_record.py \
    --csv_input /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations/test_labels.csv \
    --image_dir /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/images/train_07_04_2021/ \
    --output_path /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations/test.record


2021-04-07 17:54:47.032575: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations/train.record
2021-04-07 17:54:53.755155: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Successfully created the TFRecords: /content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/annotations/test.record


In [13]:
import tensorflow.compat.v2 as tf
from object_detection import model_lib_v2

In [ ]:
pipeline_config_path = '/content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/result_models/ssd_mobilenet_v2_320x320/pipeline.config'
model_dir = '/content/drive/MyDrive/artificial-intelligence/workspace/discharge_record/result_models/ssd_mobilenet_v2_320x320'
num_train_steps = None
eval_on_train_data = False
sample_1_of_n_eval_examples = None
sample_1_of_n_eval_on_train_examples = 5
checkpoint_dir = None
eval_timeout = 3600
use_tpu = False
tpu_name = None   
num_workers = 1
checkpoint_every_n = 1000
record_summaries = True

def main(unused_argv):
  tf.config.set_soft_device_placement(True)

  if checkpoint_dir:
    model_lib_v2.eval_continuously(
        pipeline_config_path=pipeline_config_path,
        model_dir=model_dir,
        train_steps=num_train_steps,
        sample_1_of_n_eval_examples=sample_1_of_n_eval_examples,
        sample_1_of_n_eval_on_train_examples=(
            sample_1_of_n_eval_on_train_examples),
        checkpoint_dir=checkpoint_dir,
        wait_interval=300, timeout=eval_timeout)
  else:
    if use_tpu:
      # TPU is automatically inferred if tpu_name is None and
      # we are running under cloud ai-platform.
      resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_name)
      tf.config.experimental_connect_to_cluster(resolver)
      tf.tpu.experimental.initialize_tpu_system(resolver)
      strategy = tf.distribute.experimental.TPUStrategy(resolver)
    elif num_workers > 1:
      strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
    else:
      strategy = tf.compat.v2.distribute.MirroredStrategy()

    with strategy.scope():
      model_lib_v2.train_loop(
          pipeline_config_path=pipeline_config_path,
          model_dir=model_dir,
          train_steps=num_train_steps,
          use_tpu=use_tpu,
          checkpoint_every_n=checkpoint_every_n,
          record_summaries=record_summaries)

if __name__ == '__main__':
  tf.compat.v1.app.run()

W0407 18:13:14.446571 140092804573056 cross_device_ops.py:1321] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


I0407 18:13:14.449906 140092804573056 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Maybe overwriting train_steps: None


I0407 18:13:14.470187 140092804573056 config_util.py:552] Maybe overwriting train_steps: None


...

W0407 18:14:01.782534 140092804573056 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._prediction_heads.class_predictions_with_background.5._class_predictor_layers.0.kernel


W0407 18:14:01.785195 140092804573056 util.py:161] Unresolved object in checkpoint: (root).model._box_predictor._prediction_heads.class_predictions_with_background.5._class_predictor_layers.0.bias


W0407 18:14:01.787589 140092804573056 util.py:169] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


Instructions for updating:
Use fn_output_signature instead


W0407 18:14:08.641888 140090680129280 deprecation.py:537] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/util/deprecation.py:605: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead
